# Uncapacitated Lot-Sizing (ULS)

The problem is to decide on a production plan for an \( n \)-period horizon for a single product. The basic model can be viewed as having data:

- \( f_t \) is the fixed cost of producing in period \( t \)
- \( p_t \) is the unit production cost in period \( t \)
- \( h_t \) is the unit storage cost in period \( t \)
- \( d_t \) is the demand in period \( t \)

## Decision Variables

- \( x_t \) is the amount produced in period \( t \)
- \( s_t \) is the stock at the end of period \( t \)
- \( y_t = 1 \) if production occurs in period \( t \), and \( y_t = 0 \) otherwise

To handle the fixed costs, we observe that a priori no upper bound is given on \( x_t \). Thus we either must use a very large value \( M \), or calculate an upper bound based on the problem data.

## Constraints

1. Inventory balance constraints:

$$
s_{t-1} + x_t = d_t + s_t \quad \text{for } t = 1, \ldots, n
$$

2. Production setup constraints:

$$
x_t \leq M y_t \quad \text{for } t = 1, \ldots, n
$$

where \( M \) is a sufficiently large number (e.g., \( M = \sum_{i=1}^{n} d_i \))

3. Variable domain constraints:

$$
s_0 = 0, \quad s_t, x_t \geq 0, \quad y_t \in \{0, 1\} \quad \text{for } t = 1, \ldots, n
$$

## Objective Function

Minimize the total cost (production, storage, and fixed costs):

$$
\min \sum_{t=1}^{n} p_t x_t + \sum_{t=1}^{n} h_t s_t + \sum_{t=1}^{n} f_t y_t
$$

## Complete MIP Formulation

$$
\begin{aligned}
& \min \sum_{t=1}^{n} p_t x_t + \sum_{t=1}^{n} h_t s_t + \sum_{t=1}^{n} f_t y_t \\
& \text{subject to:} \\
& \quad s_{t-1} + x_t = d_t + s_t, \quad t = 1, \ldots, n \\
& \quad x_t \leq M y_t, \quad t = 1, \ldots, n \\
& \quad s_0 = 0 \\
& \quad s_t, x_t \geq 0, \quad t = 1, \ldots, n \\
& \quad y_t \in \{0, 1\}, \quad t = 1, \ldots, n
\end{aligned}
$$

## Tightened Formulation

If we impose that \( s_n = 0 \), then we can tighten the variable upper bound constraints to:

$$
x_t \leq \left(\sum_{i=t}^{n} d_i\right) y_t \quad \text{for } t = 1, \ldots, n
$$

Note also that by substituting

$
s_t = \sum_{i=1}^{t} x_i - \sum_{i=1}^{t} d_i,
$

the objective function can be rewritten as:

$$
\sum_{t=1}^{n} c_t x_t + \sum_{t=1}^{n} f_t y_t - K
$$

where

\begin{align*}
c_t = p_t + h_t + \ldots + h_n
\end{align*}

and the constant

\begin{align*}
K = \sum_{t=1}^{n} h_t \left(\sum_{i=1}^{t} d_i \right).
\end{align*}


In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 35.2 MB/s eta 0:00:00


In [2]:
import gurobipy as gp
from gurobipy import GRB
import random

In [3]:
T = 4   # number of periods
demand = [20, 15, 25, 10]   # demand in each period
prod_cost = [3, 2, 4, 3]    # unit production cost per period
hold_cost = [1, 1, 1, 1]    # unit holding cost per period
model = gp.Model("ULS_Small")
"""Variables"""
x = model.addVars(T, name="Prod")      # production in period t
I = model.addVars(T, name="Inv")       # inventory at end of period t
"""Objective: Minimize production + holding costs"""
model.setObjective(
    gp.quicksum(prod_cost[t] * x[t] for t in range(T)) +
    gp.quicksum(hold_cost[t] * I[t] for t in range(T)),
    GRB.MINIMIZE
)
"""Constraints: inventory balance"""
for t in range(T):
  if t == 0:
    model.addConstr(x[0] - I[0] == demand[0], name=f"Bal_{t}")
  else:
    model.addConstr(I[t-1] + x[t] - I[t] == demand[t], name=f"Bal_{t}")
model.optimize()

print("Production plan:")
for t in range(T):
  print(f"  Period {t+1}: Produce {x[t].X:.1f}, End Inv {I[t].X:.1f}")
print("Total cost:", model.ObjVal, "\n")

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 4 rows, 8 columns and 11 nonzeros
Model fingerprint: 0xdaee04f7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Presolve removed 4 rows and 8 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9500000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.950000000e+02
Production plan:
  Period 1: Produce 20.0, End Inv 0.0
  Period 2: Produce 40.0, End Inv 25.0
  Period 3: Produce 0.0, End Inv 0.

In [4]:
"""DICT"""
data = {
    "periods": [1, 2, 3],
    "demand": {1: 12, 2: 18, 3: 10},
    "prod_cost": {1: 2, 2: 3, 3: 2},
    "hold_cost": {1: 1, 2: 1, 3: 1},
}
T = len(data["periods"])
model = gp.Model("ULS_Dict")
x = model.addVars(data["periods"], name="Prod")
I = model.addVars(data["periods"], name="Inv")
model.setObjective(
    gp.quicksum(data["prod_cost"][t] * x[t] for t in data["periods"]) +
    gp.quicksum(data["hold_cost"][t] * I[t] for t in data["periods"]),
    GRB.MINIMIZE
)
for t in data["periods"]:
  if t == 1:
    model.addConstr(x[t] - I[t] == data["demand"][t], name=f"Bal_{t}")
  else:
    model.addConstr(I[t-1] + x[t] - I[t] == data["demand"][t], name=f"Bal_{t}")
model.optimize()

print("Production plan:")
for t in data["periods"]:
  print(f"  Period {t}: Produce {x[t].X:.1f}, End Inv {I[t].X:.1f}")
print("Total cost:", model.ObjVal, "\n")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 3 rows, 6 columns and 8 nonzeros
Model fingerprint: 0x9dacd5f1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Presolve removed 3 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.8000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.800000000e+01
Production plan:
  Period 1: Produce 12.0, End Inv 0.0
  Period 2: Produce 18.0, End Inv 0.0
  Period 3: Produce 10.0, End Inv 0.0
Total cost: 98.0 



In [5]:
random.seed(123)
T = 8
demand = [random.randint(10, 30) for _ in range(T)]
prod_cost = [random.randint(2, 5) for _ in range(T)]
hold_cost = [1 for _ in range(T)]   # constant holding cost
model = gp.Model("ULS_Random")
x = model.addVars(T, name="Prod")
I = model.addVars(T, name="Inv")
model.setObjective(
    gp.quicksum(prod_cost[t] * x[t] for t in range(T)) +
    gp.quicksum(hold_cost[t] * I[t] for t in range(T)),
    GRB.MINIMIZE
)
for t in range(T):
  if t == 0:
    model.addConstr(x[0] - I[0] == demand[0], name=f"Bal_{t}")
  else:
    model.addConstr(I[t-1] + x[t] - I[t] == demand[t], name=f"Bal_{t}")
model.optimize()

print("Demands:", demand)
print("Production costs:", prod_cost)
print("Production plan:")
for t in range(T):
  print(f"  Period {t+1}: Produce {x[t].X:.1f}, End Inv {I[t].X:.1f}")

print("Total cost:", model.ObjVal)

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 8 rows, 16 columns and 23 nonzeros
Model fingerprint: 0x408c789d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Presolve removed 8 rows and 16 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2500000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.250000000e+02
Demands: [11, 18, 12, 23, 18, 13, 11, 22]
Production costs: [4, 4, 2, 3, 3, 4, 4, 3]
Production plan:
  Period 1: Produce 11.0, End Inv 0.0
  Period 2: Produce 18.0, End Inv 0.0
  Period 3: Produce